# UFC_Stats Web Scraping

## importing pertinent libraries

In [1]:
#importing pertinent libraries
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests, re
import json
import itertools
import dask.dataframe as dd

## Extracting UFC urls

In [2]:
#creating a list of unique fighter URLs available on website
fighter_url_list = []
response = requests.get('http://www.ufcstats.com/statistics/fighters?char=a&page=all')
soup = BeautifulSoup(response.content, 'html.parser')

#targeting URLs that contain fighter data
fighters = soup.findAll('a', attrs={'href': re.compile('http:')})
for fighter in fighters:
    fighter_url_list.append(fighter.get('href'))

#removing duplicate URLs
fighter_url_list = list(set(fighter_url_list))

In [3]:
fighter_url_list[:15]

['http://www.ufcstats.com/fighter-details/fe5a9547479396ab',
 'http://www.ufcstats.com/fighter-details/9e3dbb9c68ed5d1a',
 'http://www.ufcstats.com/fighter-details/1562b12763cc8d67',
 'http://www.ufcstats.com/fighter-details/39f62b833e4cf126',
 'http://www.ufcstats.com/fighter-details/d0f3959b4a9747e6',
 'http://www.ufcstats.com/fighter-details/063649e21bc9d6d5',
 'http://www.ufcstats.com/fighter-details/2eada40ac8801559',
 'http://www.ufcstats.com/fighter-details/7bd94b60d7521e4a',
 'http://www.ufcstats.com/fighter-details/c487223b0289bda9',
 'http://www.ufcstats.com/fighter-details/15df64c02b6b0fde',
 'http://www.ufcstats.com/fighter-details/73ef22f25d0f70e2',
 'http://www.ufcstats.com/fighter-details/6eaec40f724852f4',
 'http://www.ufcstats.com/fighter-details/87a1dc546b1c5caf',
 'http://www.ufcstats.com/statistics/fighters?char=a',
 'http://www.ufcstats.com/fighter-details/e17770faae3ca54c']

Looks like there are some URLs we don't need in out list. Let's weed them out.

In [4]:
#removing invalid URLs
for url in fighter_url_list:
    if 'fighter-details' not in url:
        fighter_url_list.remove(url)

## Creating Soup Dataframes

## Creating Dataframe Building Blocks

In [5]:
def info_generator(url):
    '''This function parses basic request to a url'''
    
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    return soup

In [6]:
#creating a soup list to reduce number of website requests
basic_soup_list = list(map(info_generator, fighter_url_list))

In [7]:
def base_stats_soup(soup):
    '''This function identifies all statistics for later extraction'''
    
    base_stats = soup.findChildren('li', attrs={'class': re.compile('b-list')})
    
    return base_stats

In [8]:
def fighter_history_fetcher(soup):
    '''This function is meant to extract the html information for each fighter as a list of urls. '''
    
    soup_holder = soup.findAll('a', attrs={'href': re.compile('fight-details')})
    
    list_of_fights = []
    
    for x in soup_holder:
        fight_holder = []
        
        fight_holder.append(x.get('href'))
        
        list_of_fights.append(fight_holder) 
        
    return list_of_fights

### Generating basic statistics lists

In [10]:
def stats_soup(soup):
    '''parses soups down to the level of stats for faster, fluid parsing'''
    
    base_stats = soup.findChildren('li', attrs={'class': re.compile('b-list')})
    
    return base_stats

In [11]:
stats_soups = list(map(stats_soup, basic_soup_list))

In [152]:
fighter_history_url_list = list(map(fighter_history_fetcher, basic_soup_list))
fighter_history_url_list[:3]

[[['http://www.ufcstats.com/fight-details/435c93cb89016fd1'],
  ['http://www.ufcstats.com/fight-details/b2857e799319f931'],
  ['http://www.ufcstats.com/fight-details/b91807add93763c9'],
  ['http://www.ufcstats.com/fight-details/f7fac3df5c6dc91f']],
 [['http://www.ufcstats.com/fight-details/cb640b7dcc9db7e8'],
  ['http://www.ufcstats.com/fight-details/431f9bad886eb2ae']],
 [['http://www.ufcstats.com/fight-details/c15b253026ff6b86'],
  ['http://www.ufcstats.com/fight-details/a0dfc397caef4afe']]]

In [153]:
def compound_flattener(fight_url_list):
    flattened_fight_urls = list(itertools.chain(*fight_url_list))
    return flattened_fight_urls

In [154]:
fighter_history_url_list = list(map(compound_flattener, fighter_history_url_list))

In [155]:
fighter_history_url_list[:3]

[['http://www.ufcstats.com/fight-details/435c93cb89016fd1',
  'http://www.ufcstats.com/fight-details/b2857e799319f931',
  'http://www.ufcstats.com/fight-details/b91807add93763c9',
  'http://www.ufcstats.com/fight-details/f7fac3df5c6dc91f'],
 ['http://www.ufcstats.com/fight-details/cb640b7dcc9db7e8',
  'http://www.ufcstats.com/fight-details/431f9bad886eb2ae'],
 ['http://www.ufcstats.com/fight-details/c15b253026ff6b86',
  'http://www.ufcstats.com/fight-details/a0dfc397caef4afe']]

### Basic Statistics DataFrame Construction

In [17]:
def fighter_name(soup):
    
    name = soup.find('span', attrs={'class': re.compile('b-content__title-highlight')})
    
    return name.text.strip()#[0].text.strip()

In [18]:
def fighter_record(soup):
    
    record = soup.findChildren('span', attrs={'class': re.compile('b-content__title-record')})
    
    return record[0].text.strip().split(':')[1].strip()

In [19]:
def basic_stats_height(soup):
    '''soups run thorugh this function should be run through stats_soup function beforehand.'''
    
    #feet
    height = soup[0].text.strip().split(':')[1].strip().split(' ')[0].replace("'", "")
    
   #inches
    if height == '--':
        pass
    else:
        
        height = int(height)
        
        inches = int(soup[0].text.strip().split(':')[1].strip().split(' ')[1].replace("'", "").replace('"', '').replace("--", "0"))

        height_to_inches = (height * 12) 
    
        final_height = height_to_inches + inches
    
        return (inches + height_to_inches)

In [20]:
def basic_stats_weight(soup):
    
    weight = soup[1].text.strip().split(':')[1].strip().replace(' lbs.', '')
    
    if weight == '--':
        pass
    else:
        weight = int(weight)
    return weight 

In [21]:
def basic_stats_reach(soup):
    
    reach = soup[2].text.strip().split(':')[1].strip().replace('"', '')
    
    if reach == '--':
        pass
    else:
        reach = int(reach)
    return reach

In [22]:
def basic_stats_stance(soup):
    
    stance = soup[3].text.strip().split(':')[1].strip()
    return stance

In [23]:
def basic_stats_DOB(soup):
    
    DOB = soup[4].text.strip().split(':')[1].strip()
    return DOB

In [24]:
fighter_names = list(map(fighter_name, basic_soup_list))

fighter_records = list(map(fighter_record, basic_soup_list))

fighter_heights =  list(map(basic_stats_height, stats_soups))

fighter_weights = list(map(basic_stats_weight, stats_soups))

fighter_reachs = list(map(basic_stats_reach, stats_soups))

fighter_stances = list(map(basic_stats_stance, stats_soups))

fighter_DOBs = list(map(basic_stats_DOB, stats_soups))

In [25]:
fighter_basic_stats_df = pd.DataFrame()

fighter_basic_stats_df['name'] = fighter_names
fighter_basic_stats_df['record'] = fighter_records
fighter_basic_stats_df['height'] = fighter_heights
fighter_basic_stats_df['weight'] = fighter_weights
fighter_basic_stats_df['reach'] = fighter_reachs
fighter_basic_stats_df['stance'] = fighter_stances
fighter_basic_stats_df['DOB'] = fighter_DOBs

In [26]:
fighter_basic_stats_df.head()

,name,record,height,weight,reach,stance,DOB
0,Jim Alers,13-3-0 (1 NC),69.0,145,71,Orthodox,"Oct 14, 1986"
1,Mike Aina,12-6-1 (1 NC),69.0,155,--,Orthodox,--
2,Alex Andrade,10-5-0,71.0,200,--,Orthodox,"May 14, 1974"
3,Juan Alcain,1-2-0,NaN,--,--,,--
4,Jose Aldo,28-6-0,67.0,135,70,Orthodox,"Sep 09, 1986"


In [27]:
fighter_basic_stats_df.to_csv('fighter_basic_stats_csv', index=False)

### Career Statistics DataFrame Construction

In [28]:
def career_stats_soup(soup):
    
    career_stats_soup = soup.findChildren('li', attrs={'class': re.compile('b-list')})
    career_stats = career_stats_soup[5:]
    
    return career_stats_soup

In [29]:
def career_stats_SLpM(soup):

    SLpM = soup[5].text.split(':')[1].replace('%', '').strip()
    
    if SLpM == '--':
        pass
    else:
        SLpM = float(SLpM)
        
    return SLpM

In [30]:
def career_stats_StrAcc(soup):
    StrAcc =  soup[6].text.split(':')[1].replace('%', '').strip()
    
    if StrAcc == '--':
        pass
    else:
        StrAcc = float(StrAcc)
        
    return StrAcc

In [31]:
def career_stats_SApM(soup):
    SApM = soup[7].text.split(':')[1].strip()
    
    if SApM == '--':
        pass
    else:
        SApM = float(SApM)
        
    return SApM

In [32]:
def career_stats_StrDef(soup):
    StrDef = soup[8].text.split(':')[1].replace('%', '').strip()
    
    if StrDef == '--':
        pass
    else:
        StrDef = float(StrDef)
        
    return StrDef

In [33]:
def career_stats_TDAvg(soup):
    TDAvg = soup[10].text.split(':')[1].strip()
    
    if TDAvg == '--':
        pass
    else:
        TDAvg = float(TDAvg)
        
    return TDAvg

In [34]:
def career_stats_TDAcc(soup):
    TDAcc = soup[11].text.split(':')[1].strip().replace('%', '')
    
    if TDAcc == '--':
        pass
    else:
        TDAcc = float(TDAcc)
        
    return TDAcc

In [35]:
def career_stats_TDDef(soup):
    TDDef = soup[12].text.split(':')[1].replace('%', '').strip()
    
    if TDDef == '--':
        pass
    else:
        TDDef = float(TDDef)
        
    return TDDef

In [36]:
def career_stats_SubAvg(soup):
    SubAvg =  soup[13].text.split(':')[1].strip().replace('%', '')  
    
    if SubAvg == '--':
        pass
    else:
        SubAvg = float(SubAvg)
        
    return SubAvg

In [37]:
fighter_SLpMs = list(map(career_stats_SLpM, stats_soups))

fighter_StrAccs = list(map(career_stats_StrAcc, stats_soups))

fighter_SApMs = list(map(career_stats_SApM, stats_soups))

fighter_StrDefs =list(map(career_stats_StrDef, stats_soups))

fighter_TDAvgs = list(map(career_stats_TDAvg, stats_soups))

fighter_TDAccs = list(map(career_stats_TDAcc, stats_soups))

fighter_TDDefs = list(map(career_stats_TDDef, stats_soups))

fighter_SubAvgs = list(map(career_stats_SubAvg, stats_soups))

In [38]:
fighter_career_stats_df = pd.DataFrame()

fighter_career_stats_df['name'] = fighter_names
fighter_career_stats_df['SLpMs'] = fighter_SLpMs
fighter_career_stats_df['StrAccs'] = fighter_StrAccs
fighter_career_stats_df['SApMs'] = fighter_SApMs
fighter_career_stats_df['StrDefs'] = fighter_StrDefs
fighter_career_stats_df['TDAvgs'] = fighter_TDAvgs
fighter_career_stats_df['TDAccs'] = fighter_TDAccs
fighter_career_stats_df['TDDefs'] = fighter_TDDefs
fighter_career_stats_df['SubAvgs'] = fighter_SubAvgs

In [39]:
fighter_career_stats_df.head()

,name,SLpMs,StrAccs,SApMs,StrDefs,TDAvgs,TDAccs,TDDefs,SubAvgs
0,Jim Alers,2.90,40.0,4.38,56.0,2.26,41.0,80.0,0.3
1,Mike Aina,2.87,33.0,6.33,59.0,0.00,0.0,71.0,0.0
2,Alex Andrade,0.20,36.0,2.60,53.0,0.00,0.0,25.0,0.8
3,Juan Alcain,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,Jose Aldo,3.47,44.0,3.18,65.0,0.64,65.0,91.0,0.1


In [40]:
fighter_career_stats_df.to_csv('fighter_career_stats_csv', index=False)

### Constructing Fight Event and Fight List Database

In [41]:
#creating a list of unique fighter URLs available on website
event_url_list = []
response = requests.get('http://www.ufcstats.com/statistics/events/completed?page=all')
soup = BeautifulSoup(response.content, 'html.parser')

#targeting URLs that contain fighter data
events = soup.findAll('a', attrs={'href': re.compile('http:')})
for event in events:
    event_url_list.append(event.get('href'))

#removing duplicate URLs
event_url_list = list(set(event_url_list))

In [48]:
#removing invalid URLs
for url in event_url_list:
    if 'event-details' not in url:
        event_url_list.remove(url)

In [50]:
event_database_df = pd.DataFrame()

event_database_df['event'] = event_url_list

In [51]:
event_soups = list(map(info_generator, event_url_list))

In [95]:
def event_fights_parser(event_soup):
        
    holder = event_soup.findAll('a', attrs={'href': re.compile('fight-details')})
    
    event_fights_holder = []
    
    for x in range(0, len(holder)):
        
        event_fights_holder.append(holder[x].get('href'))
        
    return event_fights_holder

In [100]:
event_fight_list = list(map(event_fights_parser, event_soups))

In [102]:
event_database_df['fight_list'] = event_fight_list

In [351]:
def fight_participants_fetcher(event_soup):
    
    fight_pairs = []
    
    name_holder = event_soup.findChildren('a', attrs={'class': re.compile('b-link')})

    fighter1_list_holder, fighter2_list_holder = name_holder[0::2], name_holder[1::2]

    fighter1_list = []
    fighter2_list = []

    for fighter1 in fighter1_list_holder:
        fighter1_list.append(fighter1.text.strip())
        
    for fighter2 in fighter2_list_holder:
        fighter2_list.append(fighter2.text.strip())
      
    fighters_in_ring = zip(fighter1_list, fighter2_list)
    
    for duo in fighters_in_ring:
        fight_pairs.append(duo)
        
    return fight_pairs

In [357]:
matchups = list(map(fight_participants_fetcher, event_soups))

In [358]:
event_database_df['matchups'] = matchups

In [359]:
event_database_df.head()

,event,fight_list,matchups
0,http://www.ufcstats.com/event-details/c75b9988...,[http://www.ufcstats.com/fight-details/ea64af6...,"[(Tito Ortiz, Vladimir Matyushenko), (Jens Pul..."
1,http://www.ufcstats.com/event-details/eae4aec1...,[http://www.ufcstats.com/fight-details/0bdc4e7...,"[(Cain Velasquez, Junior Dos Santos), (Daniel ..."
2,http://www.ufcstats.com/event-details/b757c73f...,[http://www.ufcstats.com/fight-details/5d7c91a...,"[(Gegard Mousasi, Mark Munoz), (CB Dollaway, F..."
3,http://www.ufcstats.com/event-details/738acab0...,[http://www.ufcstats.com/fight-details/9b9591a...,"[(Demetrious Johnson, John Dodson), (Glover Te..."
4,http://www.ufcstats.com/event-details/c933d423...,[http://www.ufcstats.com/fight-details/2ae8623...,"[(Chuck Liddell, Randy Couture), (Renato Sobra..."


In [360]:
event_database_df.to_csv('event_database_csv', index=False)

PROTOCODE ==================================